http://www.nct9.ne.jp/m_hiroi/light/pulp06.html
に基づいています

In [9]:
import math
def distance(p1, p2):
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    return math.sqrt(dx * dx + dy * dy)

# 隣接行列の生成
def make_matrix(ps):
    return [[distance(p1, p2) for p2 in ps] for p1 in ps]
import tkinter as tk
# 描画
def draw(ps, xs):
    root = tk.Tk()
    c0 = tk.Canvas(root, width = 500, height = 500)
    c0.pack()

    for i in range(len(xs)):
        for j in range(len(xs)):
            if xs[i][j].value() == 0: continue
            c0.create_line(ps[i][0], ps[i][1], ps[j][0], ps[j][1], width = 1.0)
    for i, j in ps:
        c0.create_oval(i - 5, j - 5, i + 5, j + 5, fill = 'green')
    root.mainloop()
import random
# 乱数でデータを生成
def make_data(n):
    return [(random.randrange(480) + 10, random.randrange(480) + 10) for _ in range(n)]



#リスト : 施設配置問題 (p-median)
import pulp
prob = pulp.LpProblem('p-median')
# コスト (距離)
#      f0  f1  f2
cs = [[11,  3,  6],   # d0
      [ 1,  7,  9],   # d1
      [ 8,  5, 12]]   # d2
# 人数
ws = [2, 10, 4]
# 変数
# fs[i]    : 施設 fi を配置するか否か
# xs[i][j] : di が fi を利用するか否か
#
fs = [pulp.LpVariable('f{}'.format(i), cat='Binary') for i in range(3)]
xs = []
for i in range(3):
    xs1 = [pulp.LpVariable('x{}_{}'.format(i, j), cat='Binary') for j in range(3)]
    xs.append(xs1)

# 目的関数
prob += pulp.lpSum([ws[i] * pulp.lpDot(cs[i], xs[i]) for i in range(3)])

# 制約条件
prob += pulp.lpSum(fs) == 2
for i in range(3):
    prob += pulp.lpSum(xs[i]) == 1
for i in range(3):
    for j in range(3):
        prob += xs[i][j] <= fs[j]

#
status = prob.solve()
print("Status", pulp.LpStatus[status])
print([f.value() for f in fs])
for xs1 in xs:
    print([x.value() for x in xs1])
print(prob.objective.value())

Status Optimal
[1.0, 1.0, 0.0]
[0.0, 1.0, 0.0]
[1.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
36.0


In [11]:
#リスト : p-median 問題 (乱数での配置)
import time
def solver(ps, p):
    size = len(ps)
    cs = make_matrix(ps)   # 隣接行列

    prob = pulp.LpProblem('p-median')

    # 変数の生成
    xs = []
    for i in range(size):
        xs1 = [pulp.LpVariable('x{}_{}'.format(i, j), cat='Binary') for j in range(size)]
        xs.append(xs1)
    fs = [pulp.LpVariable('f{}'.format(i), cat='Binary') for i in range(size)]

    # 目的関数
    prob += pulp.lpSum([pulp.lpDot(c, x) for c, x in zip(cs, xs)])

    # 制約条件
    prob += pulp.lpSum(fs) == p
    for x in xs:
        prob += pulp.lpSum(x) == 1
    for i in range(size):
        prob += xs[i][i] >= fs[i]        # 削除しても動作する
        for j in range(size):
            prob += xs[i][j] <= fs[j]

    s = time.time()
    status = prob.solve()
    print("Status", pulp.LpStatus[status])
    print("z", prob.objective.value())
    print(time.time() - s)
    draw(ps, xs)

# 実行
for n in [60, 80, 100, 120, 140]:
    print("-----", n, "-----")
    solver(make_data(n), n // 20)

----- 60 -----
Status Optimal
z 6910.055484473142
0.27760982513427734
----- 80 -----
Status Optimal
z 6859.768775105929
0.576725959777832
----- 100 -----
Status Optimal
z 8167.387728889581
0.9921247959136963
----- 120 -----
Status Optimal
z 8540.308586488454
1.3052330017089844
----- 140 -----
Status Optimal
z 8840.037423735872
1.994908094406128
